In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tt
import experiments_stuff
from bilinear_package.src import primitives
from bilinear_package.src.hadamard_product import generalizedApproximateHadamardProduct
from bilinear_package.src.convolution import approximateConvolution, preciseConvolution, countFourier, countInverseFourier, approximateCycleConvolution
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from newton_experiment_primitives import create_exponential_grid, create_newtonial_potential_grid

In [24]:
from bilinear_package.src.convolution import preciseCycleConvolution
from newton_experiment_primitives import interpolateTTTensor, compressTTTensor
import pandas as pd

D = 40

df_time = pd.DataFrame()
df_precision = pd.DataFrame()

for d in range(3, 6):
    for grid_side in np.logspace(8, 10, 7, base = 2, dtype = int):
        grid_size = 8 * (grid_side // 8)
        h = (2 * D) / grid_side
        tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2)
        tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2)
        tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
        print(tt.vector.from_list(tensor1))
        print(tt.vector.from_list(tensor2))
        precise = preciseCycleConvolution(tensor1, tensor2)
        precise = primitives.twoSidedCuttingTTTensor(precise, [(0, grid_side + 1) for _ in range(d)])
        precise = tt.vector.from_list(precise)
        precise_norm = tt.vector.norm(precise)
        time1 = time.time()
        answer1 = approximateCycleConvolution(tensor1, tensor2, desired_ranks=np.ones(d - 1, dtype=int) * 30, seed = 271 * d + grid_side)
        answer1 = primitives.twoSidedCuttingTTTensor(answer1, [(0, grid_side + 1) for _ in range(d)])
        time2 = time.time()
        answer2 = precise.round(0, 30)
        time3 = time.time()
        tensor1_ = tt.vector.from_list(countFourier(tensor1))
        tensor2_ = tt.vector.from_list(countFourier(tensor2))
        multifunc = tt.multifuncrs2([tensor1_, tensor2_], lambda x : x[:, 0] * x[:, 1], eps = 1e-6, verb = 0)
        answer3 = countInverseFourier(tt.vector.to_list(multifunc))
        time4 = time.time()
        df_time[f"{d}_{grid_side}"] = [time2 - time1, time3 - time2, time4 - time3]
        precision1 = tt.vector.norm(precise - tt.vector.from_list(answer1)) / precise_norm
        precision2 = tt.vector.norm(precise - answer2) / precise_norm
        precision3 = tt.vector.norm(precise - tt.vector.from_list(answer3)) / precise_norm 
        df_precision[f"{d}_{grid_side}"] = [precision1, precision2, precision3]
        df_time.to_csv("experiments_results/experiment4/time")
        df_precision.to_csv("experiments_results/experiment4/precision")
        print(f"{d}_{grid_side}")

This is a 3-dimensional tensor 
r(0)=1, n(0)=514 
r(1)=15, n(1)=514 
r(2)=15, n(2)=514 
r(3)=1 

This is a 3-dimensional tensor 
r(0)=1, n(0)=514 
r(1)=21, n(1)=514 
r(2)=21, n(2)=514 
r(3)=1 

3_256
This is a 3-dimensional tensor 
r(0)=1, n(0)=646 
r(1)=16, n(1)=646 
r(2)=16, n(2)=646 
r(3)=1 

This is a 3-dimensional tensor 
r(0)=1, n(0)=646 
r(1)=22, n(1)=646 
r(2)=22, n(2)=646 
r(3)=1 

3_322
This is a 3-dimensional tensor 
r(0)=1, n(0)=814 
r(1)=16, n(1)=814 
r(2)=16, n(2)=814 
r(3)=1 

This is a 3-dimensional tensor 
r(0)=1, n(0)=814 
r(1)=22, n(1)=814 
r(2)=22, n(2)=814 
r(3)=1 



KeyboardInterrupt: 

In [21]:
from bilinear_package.src.convolution import preciseCycleConvolution
from newton_experiment_primitives import interpolateTTTensor, compressTTTensor
import pandas as pd

D = 40

df_time = pd.DataFrame()
df_precision = pd.DataFrame()

for d in range(2, 6):
    for grid_side in np.logspace(6, 10, 5, base = 2, dtype = int):
        tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2)
        tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2)
        tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
        precise = preciseCycleConvolution(tensor1, tensor2)
        precise = primitives.twoSidedCuttingTTTensor(precise, [(0, grid_side + 1) for _ in range(d)])
        precise = tt.vector.from_list(precise)
        precise_norm = tt.vector.norm(precise)
        answer2 = precise.round(0, 30)
        precision2 = tt.vector.norm(precise - answer2) / precise_norm
        print(precision2)

6.755760745546992e-16
9.048028347785842e-16
1.1514606305591267e-15
7.465433335647055e-15
3.5189155918100055e-15
3.2007976440729053e-15
3.503849306878575e-15


KeyboardInterrupt: 